In [1]:
from keras_preprocessing.image import img_to_array, load_img

In [2]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rescale=1/255.0,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='reflect')

generator = datagen.flow_from_directory(
    r"D:\Scrap works\Naren SR\Data assig3\flowers",
    target_size=(150,150),
    batch_size=16,
    class_mode='sparse'
)

Found 4317 images belonging to 5 classes.


In [3]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import TensorBoard
import time
%load_ext tensorboard

In [4]:
dense_layers = [1]
layer_sizes = [64]
conv_layers = [3]
SIZE = 150
INPUT_SHAPE = (SIZE, SIZE, 3) 

for dense_layer in dense_layers:
  for layer_size in layer_sizes:
    for conv_layer in conv_layers:
      NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
      tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
      
      print(NAME)
      #model initialzation
      model = Sequential()

      #model definition
      model.add(Conv2D(layer_size, (3,3), input_shape = INPUT_SHAPE))
      model.add(Activation('relu'))  
      model.add(MaxPooling2D(pool_size=(2,2)))

      for l in range(conv_layer-1):
        model.add(Conv2D(layer_size, (3,3)))
        model.add(Activation('relu')) 
        model.add(MaxPooling2D(pool_size=(2,2)))
      
      model.add(Flatten())

      for l in range(dense_layer):
        model.add(Dense(layer_size))
        model.add(Activation('relu')) 


      #final output layer
      model.add(Dense(5))
      model.add(Activation('softmax'))

      #settings
      model.compile(loss='sparse_categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])
      #fit
      model.fit(generator, batch_size=64, epochs=25, callbacks=[tensorboard])

3-conv-64-nodes-1-dense-1668764951
Epoch 1/25
270/270 [==============================] - 71s 229ms/step - loss: 1.2635 - accuracy: 0.4496
Epoch 2/25
270/270 [==============================] - 25s 94ms/step - loss: 1.0079 - accuracy: 0.6004
Epoch 3/25
270/270 [==============================] - 24s 89ms/step - loss: 0.9659 - accuracy: 0.6145
Epoch 4/25
270/270 [==============================] - 24s 90ms/step - loss: 0.8993 - accuracy: 0.6505
Epoch 5/25
270/270 [==============================] - 24s 91ms/step - loss: 0.8545 - accuracy: 0.6676
Epoch 6/25
270/270 [==============================] - 25s 92ms/step - loss: 0.8328 - accuracy: 0.6780
Epoch 7/25
270/270 [==============================] - 25s 93ms/step - loss: 0.8129 - accuracy: 0.6854
Epoch 8/25
270/270 [==============================] - 25s 94ms/step - loss: 0.7847 - accuracy: 0.6984
Epoch 9/25
270/270 [==============================] - 26s 95ms/step - loss: 0.7606 - accuracy: 0.7079
Epoch 10/25
270/270 [=========================

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 64)      1792      
_________________________________________________________________
activation (Activation)      (None, 148, 148, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
activation_1 (Activation)    (None, 72, 72, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        3

In [6]:
model.save('Flowers.h5')

In [7]:
import cv2

CATEGORIES = ['Daisy', 'Dandelion', 'Rose', 'Sunflower', 'Tulip']

def prepare(filepath):
  IMG_SIZE = 150
  img_array = cv2.imread(filepath)
  new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
  return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [17]:
prediction = model.predict([prepare(r"D:\Scrap works\Naren SR\Data assig3\flowers\daisy\21652746_cc379e0eea_m.jpg")])
print(CATEGORIES[int(prediction[0][0])])

Daisy
